In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from skimage.feature import hessian_matrix_det
from experiment_class_surf import Experiment,clean_exp_with_hyphaes
from hyphae_id_surf import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends,solve_degree4, clean_obvious_fake_tips


In [2]:
plate = 11

***Skeletonization***

In [25]:
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
len(list_dir_interest)

53

In [26]:
for j in range(0,14):
    my_file = open("/scratch/shared/mrozemul/Fiji.app/extract_skel.sh", "w")
    my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 2:00:00\n#SBATCH -p normal\n")
    my_file.write(f'for i in `seq {4*j} {4*j+3}`; do\n')
    my_file.write(f'\t python ~/Corentin/MscThesis/extract_skel_surf.py $i {plate} &\n')
    my_file.write('done\n')
    my_file.write('wait\n')
    my_file.close()
# !srun -t 2:00:00 --pty /scratch/shared/mrozemul/Fiji.app/extract_skel.sh
    !sbatch /scratch/shared/mrozemul/Fiji.app/extract_skel.sh

Submitted batch job 9377516
Submitted batch job 9377517
Submitted batch job 9377518
Submitted batch job 9377519
Submitted batch job 9377520
Submitted batch job 9377521
Submitted batch job 9377522
Submitted batch job 9377523
Submitted batch job 9377524
Submitted batch job 9377525
Submitted batch job 9377526
Submitted batch job 9377527
Submitted batch job 9377528
Submitted batch job 9377529


***Check Skeletonization***

In [ ]:
plt.close('all')
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
begin = 1
end = 50
dates_datetime_chosen=dates_datetime[begin:end]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
dilateds=[]
skels = []
skel_docs = []
Rs=[]
ts=[]
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton.mat')
    skel = skel_info['skeleton']
    skels.append(skel)
    skel_doc = sparse_to_doc(skel)
    skel_docs.append(skel_doc)
#     transform = sio.loadmat(path_snap+'/Analysis/transform.mat')
#     R,t = transform['R'],transform['t']
#     Rs.append(R)
#     ts.append(t)
start = 25
for j in range(start,start + 10):
    print(dates[j-1],j-1+begin)
    skeleton1,skeleton2 = skel_docs[j-1],skel_docs[j]
#     R,t = Rs[j],ts[j]
    skelet_pos = np.array(list(skeleton1.keys()))
    samples = np.random.choice(skelet_pos.shape[0],20000)
    X = np.transpose(skelet_pos[samples,:])
    skelet_pos = np.array(list(skeleton2.keys()))
    samples = np.random.choice(skelet_pos.shape[0],20000)
    Y = np.transpose(skelet_pos[samples,:])
    fig=plt.figure(figsize=(10,9))
    ax = fig.add_subplot(111)
#     Yrep=np.transpose(np.transpose(np.dot(R,X))+t)
#     ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
    ax.scatter(np.transpose(X)[:,0],np.transpose(X)[:,1])
    ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])

***Check specific image***

In [21]:
i = 25
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
date =dates [i]
directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
path_tile=path_snap+'/Img/TileConfiguration.txt.registered'
try:
    tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
except:
    print('error_name')
    path_tile=path_snap+'/Img/TileConfiguration.registered.txt'
    tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
t=time()
xs =[c[0] for c in tileconfig[2]]
ys =[c[1] for c in tileconfig[2]]
dim = (int(np.max(ys)-np.min(ys))+4096,int(np.max(xs)-np.min(xs))+4096)
ims = []
for name in tileconfig[0]:
#     ims.append(imageio.imread('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+name))
    ims.append(imageio.imread(f'{name}'))


In [ ]:
plt.close('all')
shape = (3000,4096)
linex  = 12500
liney = 10000
for index,im in enumerate(ims):
    boundaries = int(tileconfig[2][index][0]-np.min(xs)),int(tileconfig[2][index][1]-np.min(ys))
    if boundaries[1]<=linex< boundaries[1]+shape[0] and boundaries[0]<=liney< boundaries[0]+shape[1]:
        print(index)
        im_cropped=im
        im_blurred =cv2.blur(im_cropped, (200, 200))
        im_back_rem = (im_cropped+1)/(im_blurred+1)*120
        # # im_back_rem = im_cropped*1.0
        # # # im_back_rem = cv2.normalize(im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
        frangised = frangi(im_back_rem,sigmas=range(1,20,4))*255
        # # frangised = cv2.normalize(frangised, None, 0, 255, cv2.NORM_MINMAX)
        hessian = hessian_matrix_det(im_back_rem,sigma = 20)
        transformed = (frangised+cv2.normalize(abs(hessian), None, 0, 255, cv2.NORM_MINMAX)-im_back_rem+120)*(im_blurred>=35)
        low = 20
        high = 100
        lowt = (transformed > low).astype(int)
        hight = (transformed > high).astype(int)
        hyst = filters.apply_hysteresis_threshold(transformed, low, high)
        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(111)
        ax.imshow(im)
        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(111)
        ax.imshow(frangised)
        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(111)
        ax.imshow(transformed)
        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(111)
        ax.imshow(hyst)
        
        

***Realign***

In [10]:
for j in range(0,7):
    my_file = open("/scratch/shared/mrozemul/Fiji.app/align.sh", "w")
    my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 2:00:00\n#SBATCH -p normal\n")
    my_file.write(f'for i in `seq {8*j} {8*j+7}`; do\n')
    my_file.write(f'\t python ~/Corentin/MscThesis/final_alignment.py $i {plate} &\n')
    my_file.write('done\n')
    my_file.write('wait\n')
    my_file.close()
# !srun -t 2:00:00 --pty /scratch/shared/mrozemul/Fiji.app/align.sh
    !sbatch /scratch/shared/mrozemul/Fiji.app/align.sh

Submitted batch job 9377983
Submitted batch job 9377984
Submitted batch job 9377985
Submitted batch job 9377986
Submitted batch job 9377987
Submitted batch job 9377988
Submitted batch job 9377989


***Check Alignment***

In [ ]:
plt.close('all')
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
begin = 1
end = 35
dates_datetime_chosen=dates_datetime[begin:end]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
dilateds=[]
skels = []
skel_docs = []
directory_name=f'2020{dates[0]}_Plate{0 if plate<10 else ""}{plate}'
path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
skel_info = read_mat(path_snap+'/Analysis/skeleton.mat')
skel = skel_info['skeleton']
skels.append(skel)
skel_doc = sparse_to_doc(skel)
skel_docs.append(skel_doc)
Rs=[]
ts=[]
for date in dates[1:]:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton.mat')
    skel = skel_info['skeleton']
    skels.append(skel)
    skel_doc = sparse_to_doc(skel)
    skel_docs.append(skel_doc)
    transform = sio.loadmat(path_snap+'/Analysis/transform.mat')
    R,t = transform['R'],transform['t']
    Rs.append(R)
    ts.append(t)
start = 15
for j in range(start,start + 10):
    print(dates[j-1],j-1+begin)
    skeleton1,skeleton2 = skel_docs[j],skel_docs[j+1]
    R,t = Rs[j],ts[j]
    skelet_pos = np.array(list(skeleton1.keys()))
    samples = np.random.choice(skelet_pos.shape[0],20000)
    X = np.transpose(skelet_pos[samples,:])
    skelet_pos = np.array(list(skeleton2.keys()))
    samples = np.random.choice(skelet_pos.shape[0],20000)
    Y = np.transpose(skelet_pos[samples,:])
    fig=plt.figure(figsize=(10,9))
    ax = fig.add_subplot(111)
    Yrep=np.transpose(np.transpose(np.dot(R,X))+t)
    ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
    ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])

***Create realigned Skeleton***

In [3]:
begin = 0
end = 35
my_file = open("/scratch/shared/mrozemul/Fiji.app/align_skelt.sh", "w")
my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 2:00:00\n#SBATCH -p normal\n")
my_file.write(f'python ~/Corentin/MscThesis/realign_surf.py {plate} {begin} {end}&\n')
my_file.write('wait\n')
my_file.close()
# !srun -t 2:00:00 --pty /scratch/shared/mrozemul/Fiji.app/align_skelt.sh
!sbatch /scratch/shared/mrozemul/Fiji.app/align_skelt.sh

Submitted batch job 9378075


***Check Fine Alignment***

In [ ]:
plt.close('all')
zone = (14000,18000,2500,7500)
skels_aligned = []
begin = 10
end = 20
for i,skel in enumerate(skel_docs[:35]):
    directory_name=f'2020{dates[i]}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    print('treatin',i)
    skels_aligned.append(sio.loadmat(path_snap+'/Analysis/skeleton_realigned.mat')['skeleton'])
for i in range(begin,end):
    plot_t_tp1([],[],None,None,skels_aligned[i][zone[0]:zone[1],zone[2]:zone[3]].todense(),skels_aligned[i+1][zone[0]:zone[1],zone[2]:zone[3]].todense())

***Prune Graph***

In [4]:
for j in range(0,1):
    my_file = open("/scratch/shared/mrozemul/Fiji.app/nx_graph_extr.sh", "w")
    my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 20:00\n#SBATCH -p normal\n")
    my_file.write(f'for i in `seq {40*j} {40*j+39}`; do\n')
    my_file.write(f'\t python ~/Corentin/MscThesis/extract_nx_graph.py {plate} $i &\n')
    my_file.write('done\n')
    my_file.write('wait\n')
    my_file.close()
    !sbatch /scratch/shared/mrozemul/Fiji.app/nx_graph_extr.sh
# !srun -t 2:00:00 --pty /scratch/shared/mrozemul/Fiji.app/nx_graph_extr.sh


Submitted batch job 9377277


***Check Pruned Graphs***

In [21]:
begin = 8
end = 16
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[begin:end]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
nx_graph_pos = []
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    path_save = path_snap +'/Analysis/nx_graph_pruned.p'
    nx_graph_pos.append(pickle.load(open( path_save, "rb" )))
skeletons = [generate_skeleton(c[0],(26322, 49527)) for c in nx_graph_pos]
compressed = [compress_skeleton(skeleton,5) for skeleton in skeletons]

In [ ]:
plt.close('all')
for i in range(len(compressed)-1):
    plot_t_tp1([],[],None,None,compressed[i],compressed[i+1])

***Identify Nodes***

In [6]:
begin = 0
end = 35
my_file = open("/scratch/shared/mrozemul/Fiji.app/extract_nodes.sh", "w")
my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 4:00:00\n#SBATCH -p normal\n")
my_file.write(f'python ~/Corentin/MscThesis/extract_nodes_surf.py {plate} {begin} {end}&\n')
my_file.write('wait\n')
my_file.close()
!sbatch /scratch/shared/mrozemul/Fiji.app/extract_nodes.sh
# !srun -t 2:00:00 --pty /scratch/shared/mrozemul/Fiji.app/extract_nodes.sh

Submitted batch job 9377266


***Check Node Id***

In [3]:
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[0:35]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
exp= Experiment(plate)
exp.load(dates)

In [10]:
plt.close('all')
t = 1
nodes = np.random.choice(exp.nx_graph[t].nodes,100)
exp.plot([t,t+1,t+5],[list(nodes)]*3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Hyphae extratcion***

In [4]:
begin = 0
end = 20
my_file = open("/scratch/shared/mrozemul/Fiji.app/extract_hyphae.sh", "w")
my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 1:00:00\n#SBATCH -p normal\n")
my_file.write(f'python ~/Corentin/MscThesis/hyphae_extraction.py {plate} {begin} {end}&\n')
my_file.write('wait\n')
my_file.close()
# !srun -t 2:00:00 --pty /scratch/shared/mrozemul/Fiji.app/extract_hyphae.sh
!sbatch /scratch/shared/mrozemul/Fiji.app/extract_hyphae.sh

Submitted batch job 9378076
